In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [6]:
data = pd.read_csv("/content/drive/My Drive/finalll.csv")



In [5]:

import pandas as pd
import numpy as np

In [7]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
data.head()

,comment_text,obsessive
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [10]:
def clean(text):
    # fill the missing entries and convert them to lower case
    text = text.fillna("fillna").str.lower()
    # replace the newline characters with space 
    text = text.map(lambda x: re.sub('\\n',' ',str(x)))
    text = text.map(lambda x: re.sub("\[\[User.*",'',str(x)))
    # remove usernames and links
    text = text.map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    text = text.map(lambda x: re.sub("\(http://.*?\s\(http://.*\)",'',str(x)))
    text = text.map(lambda x:re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(x)))
    return text

In [11]:
data['comment_text']=clean(data['comment_text'])

In [12]:
data

,comment_text,obsessive
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m s...,0
2,hey man i m really not trying to edit war it...,0
3,more i can t make any real suggestions on im...,0
4,you sir are my hero any chance you remember...,0
...,...,...
106592,lady banned from kentucky mall jcpenny k...,1
106593,omfg i m offended i m a mailbox and i m pr...,1
106594,you don t have the balls to hashtag me as ...,1
106595,makes you ask yourself who am i then am i a...,1


In [13]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [14]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [15]:
reviews = data['comment_text']

In [16]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [17]:
Y = data['obsessive']

In [18]:
reviews_with_len = [[review, Y[i], len(review)] for i, review in enumerate(tokenized_reviews)]

In [19]:

reviews_with_len.sort(key=lambda x: x[2])

In [20]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [21]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [22]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [23]:
next(iter(batched_dataset))


(<tf.Tensor: shape=(32, 1), dtype=int32, numpy=
 array([[    0],
        [    0],
        [    0],
        [    0],
        [    0],
        [    0],
        [ 4283],
        [25380],
        [ 9385],
        [ 4283],
        [ 5511],
        [ 2831],
        [26803],
        [ 2054],
        [25416],
        [ 2773],
        [ 2995],
        [ 4863],
        [ 4474],
        [ 7592],
        [ 4283],
        [11081],
        [ 7743],
        [22418],
        [ 7743],
        [22418],
        [22418],
        [ 7743],
        [11669],
        [11669],
        [16842],
        [ 7743]], dtype=int32)>, <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 0, 0, 1, 1], dtype=int32)>)

In [24]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [42]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [35]:
vocabulary_size = len(tokenizer.vocab)
embedding_dimensions = 200
cnn_filters = 100
dnn_units = 256
model_output_classes = 2

dropout_rate = 0.2

NB_EPOCHS = 5

In [44]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [45]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [ ]:
results = text_model.evaluate(test_data)
print(results)

InvalidArgumentError: ignored

In [51]:
text_model.fit(train_data,epochs=NB_EPOCHS)

Epoch 1/5
2999/2999 [==============================] - 240s 80ms/step - loss: 0.1754 - accuracy: 0.9308
Epoch 2/5
2999/2999 [==============================] - 182s 61ms/step - loss: 0.1171 - accuracy: 0.9536
Epoch 3/5
2999/2999 [==============================] - 182s 61ms/step - loss: 0.0603 - accuracy: 0.9784
Epoch 4/5
2999/2999 [==============================] - 181s 60ms/step - loss: 0.0302 - accuracy: 0.9901
Epoch 5/5
2999/2999 [==============================] - 182s 61ms/step - loss: 0.0202 - accuracy: 0.9928


In [56]:
text_model.save('custom_model.h5py')

INFO:tensorflow:Assets written to: custom_model.h5py/assets


INFO:tensorflow:Assets written to: custom_model.h5py/assets


In [27]:
s = "You beloved person lovely "
t_reviews = [tokenize_reviews(s)]

# BATCH_SIZE = 64
# inp = pdat.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [28]:
a = tf.keras.preprocessing.sequence.pad_sequences(
    t_reviews, maxlen=64, dtype='int32', padding='pre', truncating='pre',
    value=0)

   

In [29]:
print(a)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
   2017 11419  2711  8403]]


In [ ]:
if 3.1520472e-07<0.3:
  print(1)

1


In [5]:
import pickle
from sklearn.externals import joblib

In [7]:
filename = 'tokeizer.pkl'
joblib.dump(tokenizer,filename)

['tokeizer.pkl']

In [63]:
text_model.save('/content/drive/My Drive/Model')

INFO:tensorflow:Assets written to: /content/drive/My Drive/Model/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Model/assets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [9]:
tkn = joblib.load('tokeizer.pkl')

In [ ]:
padseq = joblib.load('pad.pkl')

In [25]:
from tensorflow.keras.models import model_from_json